# Evaluación de los algoritmos de agrupar

Tenemos 3 propuestas, donde cada una tiene un total de 10 clusters: la primera toma como base un vector con las palabras clave por vídeos utilizando el algoritmo de k-means, la segunda lo hace por videos pero utilizando el espectro y k-mean, y el último intento lo hace utilizando embeddings generados por doc2vec tomando como oración el título de cada vídeo y aplicando kmeans

In [ ]:
import pickle
import json
#Para el corpus
import sys
sys.path.append('..')
from src.data import Data 
#Para calcular TF-IDF
from src.tfidf import TFIDF
from tqdm import tqdm
from operator import itemgetter
# nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords #Listas de stopwords
from collections import Counter

### Cargamos los clusters de cada intento

In [4]:
def load(path):
    """
    Carga clusters, regresa los id y documentos
    de cada cluster

    Args:
        path (str): ruta del archivo
    """
    with open(path, "rb") as f:
        model = pickle.load(f)
    return model[0], model[1]

In [5]:
#Por vídeo utilizando espectro
named_videoKM, clusters_videoKM = load("../pkl/video_classKMeans.pkl")
#Por vídeo utilizando espectro y k-means
named_videoS, clusters_videoS = load("../pkl/video_classSpectre.pkl")
#Por tpitulo utilizando doc2vec y k-means
named_videoD2V, clusters_videoD2V = load("../pkl/video_classD2V.pkl")

#### Intento 1

De los canales de los que se tomaron los videos sabemos que uno de ellos es de comida y otro trata temas políticos, por lo que como una primera prueba podemos ver si los vídeos de esos canales quedaron en los mismos clusters (sería de alguna manera verificar la pureza de 2 clusters)

Obtenemos los id de los videos de los canales mencionados:

|           id            |    canal      |
|-------------------------|---------------|
|UCJQQVLyM6wtPleV4wFBK06g | visualpolitik |
|UC_Zc2fmbDpu_arkwvCDcX5g | cocina        |

In [6]:
clusters_goal = {'cocina':[], 'política': []}
files = [('cocina','UC_Zc2fmbDpu_arkwvCDcX5g'), ('política','UCJQQVLyM6wtPleV4wFBK06g')]
for name, file in files:
    with open("../corpus/data/"+file+".json", 'rb') as f:
        chanel = json.loads(f.read().decode('utf-8', 'replace'))
        for video in chanel:
            clusters_goal[name].append(video["id"])

In [5]:
print(clusters_goal['cocina'][:10])

['IAHQXsLS3P8', 'dhYWMdHVvGg', 'ULle2vGte8k', 'gun3ouyEoIc', 'H2L_btIXzzc', 'r5IM3yYTQoU', 'ld2pqKmEdDg', 'aNEHF2g0Bos', 'GbQOvcv1cTU', 'DUhvKm4VLeg']


In [6]:
def check_goal_clusters(named_video):
    """
    Muestra en pantalla la distribución de los vídeos contenidos
    en los clusters meta
    
    Args:
        named_video (dict): diccionario con el nombre del clusters y el id de los videos
    """
    resp = [[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0]]
    for i, name in enumerate(clusters_goal):
        for id in clusters_goal[name]:
            for c, docs in named_video.items():
                if id in docs:
                    resp[i][c] += 1
    print("""Distribución de videos en los clusters
         Cocina:
            {}
        Política:
            {}""".format(resp[0], resp[1]))

Lo intentamos para los videos utilizando k-means

In [7]:
check_goal_clusters(named_videoKM)


Distribución de videos en los clusters
         Cocina:
            [40, 0, 129, 0, 1, 74, 7, 0, 0, 1]
        Política:
            [1, 0, 13, 0, 1, 694, 0, 0, 1, 0]


In [8]:
len(named_videoKM[5])

1698

Con lo anterior podemos tomar que el cluster 2 es el que corresponde a Cocina y que el cluster 5 a política, aunque el cluster 5 cuenta con 1698 videos de los que 694 son de política y 74 de cocina, lo que deja 930 vídeos que podrían ser de política o podrían ser un error al momento de agrupar

Lo intentamos ahora con los vídeos utilizando el espectro

In [9]:
check_goal_clusters(named_videoS)

Distribución de videos en los clusters
         Cocina:
            [29, 14, 24, 20, 4, 24, 31, 22, 84, 0]
        Política:
            [52, 44, 63, 50, 31, 45, 75, 31, 319, 0]


Notemos que aquí los videos de ambas categorías están distribuidos a lo largo de todos los clusters, aunque podríamo decir que el cluster 8 es el de política

Lo intentamos con los vectores de doc2vec

In [10]:
check_goal_clusters(named_videoD2V)

Distribución de videos en los clusters
         Cocina:
            [17, 22, 8, 3, 19, 0, 33, 61, 45, 61]
        Política:
            [89, 8, 4, 9, 216, 0, 170, 23, 104, 87]


Donde al igual que el anterior están distribuidos a los largo de todos los clusters

#### Intento 2

Luego podemos extraer las principales palabras de los clusters para ver si tienen relación y de verdad representan un grupo

In [6]:
#Cargamos el corpus limpio
videos = Data("../pkl/clean_videos.pkl")
videos.get_all_subtitles()
#Creamos el objeto tfidf
tfidf_obj = TFIDF(videos)
# Cargamos los valores tfidf previamente calculados
tfidf_obj.tfidf = "../pkl/tfidf.pkl" #Vectores por texto

100%|██████████| 9/9 [00:00<00:00, 15.71it/s]


In [47]:
more = ["si", "bien", "ahora", "así", "aquí", "pues", 'solo', 'vez', 'cómo',
 'bueno', 'vamos', 'ser', 'ver', 'tan', 'puede', 'puedo']
stopwords_list = stopwords.words('spanish') + more

#Cargamos tfidf por documento y obtenemos la palabra clave de cada uno
#las agrupamos por clusters
def get_keywords_clusters(named_video):
    '''
    Regresa un diccionario con los subtítulos y su vector tfidf
    '''
    
    keyword = [[],[],[],[],[],[],[],[],[],[]]
    #Iteramos sobre los canales
    for c, docs in named_video.items():
        for id in docs:
            if id in tfidf_obj.tfidf.keys():
                keyword[c] += [w for w in list(tfidf_obj.tfidf[id].keys()) if w not in stopwords_list]
        keyword[c] = sorted(Counter(keyword[c]).items(), key=itemgetter(1), reverse=True)[15:25]
    return keyword


In [48]:
keyword_kmeans = get_keywords_clusters(named_videoKM)
for i, val in enumerate(keyword_kmeans):
    print("Cluster: {}\n Palabras clave: {}\n----------------".format(i, val))

Cluster: 0
 Palabras clave: [('después', 69), ('creo', 67), ('momento', 67), ('quiero', 66), ('mundo', 66), ('menos', 65), ('decir', 64), ('cosas', 62), ('gracias', 62), ('lugar', 61)]
----------------
Cluster: 1
 Palabras clave: [('favor', 14), ('mundo', 14), ('hace', 13), ('casa', 13), ('quieres', 13), ('gracias', 13), ('entonces', 12), ('nunca', 12), ('tiempo', 11), ('dónde', 11)]
----------------
Cluster: 2
 Palabras clave: [('debe', 163), ('forma', 161), ('vemos', 160), ('día', 159), ('pueden', 159), ('casa', 156), ('va', 155), ('minutos', 155), ('decir', 154), ('mientras', 152)]
----------------
Cluster: 3
 Palabras clave: [('hola', 12), ('parece', 12), ('trabajo', 12), ('mal', 12), ('dos', 12), ('lugar', 11), ('mundo', 11), ('creo', 11), ('alguien', 11), ('favor', 11)]
----------------
Cluster: 4
 Palabras clave: [('hoy', 19), ('creo', 19), ('sé', 19), ('gracias', 19), ('alguien', 18), ('tener', 17), ('verdad', 17), ('música', 17), ('mundo', 17), ('dos', 17)]
----------------
Cl

In [49]:
keyword_S = get_keywords_clusters(named_videoS)
for i, val in enumerate(keyword_S):
    print("Cluster: {}\n Palabras clave: {}\n----------------".format(i, val))

Cluster: 0
 Palabras clave: [('vida', 150), ('día', 149), ('todas', 148), ('luego', 145), ('va', 144), ('verdad', 143), ('puedes', 141), ('forma', 139), ('parece', 139), ('entonces', 138)]
----------------
Cluster: 1
 Palabras clave: [('gracias', 78), ('día', 77), ('vídeo', 77), ('hecho', 76), ('verdad', 76), ('siempre', 76), ('tener', 76), ('forma', 73), ('después', 70), ('entonces', 70)]
----------------
Cluster: 2
 Palabras clave: [('luego', 114), ('siempre', 113), ('vídeo', 112), ('toda', 112), ('mejor', 111), ('día', 111), ('años', 108), ('mundo', 108), ('forma', 107), ('sólo', 105)]
----------------
Cluster: 3
 Palabras clave: [('cada', 74), ('hecho', 74), ('luego', 72), ('entonces', 71), ('cosas', 70), ('siempre', 69), ('años', 69), ('tal', 68), ('caso', 68), ('muchas', 67)]
----------------
Cluster: 4
 Palabras clave: [('día', 57), ('cosas', 55), ('forma', 55), ('momento', 54), ('personas', 54), ('tener', 53), ('dos', 52), ('sido', 52), ('verdad', 51), ('incluso', 51)]
--------

In [50]:
keyword_D2V = get_keywords_clusters(named_videoD2V)
for i, val in enumerate(keyword_D2V):
    print("Cluster: {}\n Palabras clave: {}\n----------------".format(i, val))

Cluster: 0
 Palabras clave: [('tiempo', 170), ('sólo', 170), ('momento', 169), ('vídeo', 169), ('mejor', 166), ('toda', 161), ('vida', 160), ('todas', 158), ('tener', 156), ('luego', 155)]
----------------
Cluster: 1
 Palabras clave: [('parece', 81), ('hecho', 81), ('momento', 80), ('verdad', 79), ('puedes', 78), ('mismo', 77), ('gracias', 76), ('menos', 75), ('lugar', 74), ('cuenta', 73)]
----------------
Cluster: 2
 Palabras clave: [('luego', 116), ('mejor', 116), ('hecho', 114), ('momento', 113), ('verdad', 111), ('cada', 109), ('todas', 105), ('lugar', 105), ('pueden', 104), ('menos', 104)]
----------------
Cluster: 3
 Palabras clave: [('día', 271), ('menos', 268), ('parte', 267), ('mismo', 265), ('va', 264), ('quiero', 263), ('cada', 260), ('tener', 254), ('después', 253), ('momento', 251)]
----------------
Cluster: 4
 Palabras clave: [('cosas', 264), ('dos', 259), ('tiempo', 256), ('sólo', 254), ('parte', 252), ('todas', 251), ('toda', 251), ('amigos', 248), ('mismo', 246), ('eje

Hace falta hacer quitar los verbos y para dejar a las palabras representativas

#### Intento 3

También podemos comparar entre los diferentes clusters de cada intento para ver si coinciden algunos

In [51]:
#Hacemos una intersección de clusters
inter = [0,0,0,0,0,0,0,0,0,0]
for c, docs in named_videoKM.items():
    for id in docs:
        if id in named_videoD2V[c]:
            inter[c] += 1
    inter[c] = {'C': c, 
                'Total video KM': len(docs),
                'Total video S': len(named_videoS[c]),
                'Compartidos': inter[c]}

In [49]:
inter

[{'C': 0, 'Total video KM': 219, 'Total video S': 296, 'Compartidos': 20},
 {'C': 1, 'Total video KM': 84, 'Total video S': 134, 'Compartidos': 10},
 {'C': 2, 'Total video KM': 376, 'Total video S': 192, 'Compartidos': 19},
 {'C': 3, 'Total video KM': 61, 'Total video S': 143, 'Compartidos': 10},
 {'C': 4, 'Total video KM': 98, 'Total video S': 103, 'Compartidos': 9},
 {'C': 5, 'Total video KM': 1698, 'Total video S': 210, 'Compartidos': 20},
 {'C': 6, 'Total video KM': 188, 'Total video S': 313, 'Compartidos': 7},
 {'C': 7, 'Total video KM': 23, 'Total video S': 302, 'Compartidos': 3},
 {'C': 8, 'Total video KM': 230, 'Total video S': 1471, 'Compartidos': 18},
 {'C': 9, 'Total video KM': 299, 'Total video S': 112, 'Compartidos': 14}]